In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyreadstat
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [2]:
# calling biomarkers
bloodbiomk1_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\bloodbiomk1_20231117.csv"
bloodbiomk1 = pd.read_csv(bloodbiomk1_file_path)
bloodbiomk1 = pd.DataFrame(bloodbiomk1)

bloodbiomk2_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\bloodbiomk2_20231117.csv"
bloodbiomk2 = pd.read_csv(bloodbiomk2_file_path)
bloodbiomk2 = pd.DataFrame(bloodbiomk2)

csfbiomk1_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\csfbiomk1_20231117.csv"
csfbiomk1 = pd.read_csv(csfbiomk1_file_path)
csfbiomk1 = pd.DataFrame(csfbiomk1)

csfbiomk2_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\csfbiomk2_20231117.csv"
csfbiomk2 = pd.read_csv(csfbiomk2_file_path)
csfbiomk2 = pd.DataFrame(csfbiomk2)

csfbiomk3_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\csfbiomk3_20231117.csv"
csfbiomk3 = pd.read_csv(csfbiomk3_file_path)
csfbiomk3 = pd.DataFrame(csfbiomk3)



In [3]:
import os
main_path = r'Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers'
folder_name = 'blood_biomk1_to_briefa'  
save_path = os.path.join(main_path, folder_name)
os.makedirs(save_path, exist_ok=True)

# save_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\blood_biomk1_briefa" # Update this path to your desired folder


In [4]:
# Check for null values in each column
csfbiomk1_nulls = csfbiomk1.isnull().sum()

csfbiomk1_nulls_df = pd.DataFrame({
    'Column': csfbiomk1_nulls.index,
    'Number of Nulls': csfbiomk1_nulls.values
})

csfbiomk1_nulls_df_transposed = csfbiomk1_nulls_df.T

print("Null values in csfbiomk1:")
print(csfbiomk1_nulls_df)
csfbiomk1_nulls_df_transposed.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\csfbiomk1_nulls.csv", index=False)


Null values in csfbiomk1:
                           Column  Number of Nulls
0                      subject_id                0
1                           visit                0
2                     checkin_bin                0
3                     exposurebin                0
4                      age_decade                0
5                racecat_combined                0
6                        eduyears                0
7                      totyr_foot               56
8                     chiiseas_pf               56
9                      chiiyrs_pf               56
10                    chiiseas_pl               56
11                     chiiyrs_pl               56
12                    chiiseas_pg               56
13                     chiiyrs_pg               56
14     c_strem2_FLAG_insufficient               53
15  c_PDGFRbeta_FLAG_insufficient               53
16                       c_strem2               55
17                    c_PDGFRbeta               56
18   

In [5]:
# Calculate the mean for each column, ignoring NaNs
# Replace nulls with label means

for column in csfbiomk1.columns:
    if column != 'checkin_bin':  # Skip the label column

        averages = csfbiomk1.groupby('checkin_bin')[column].mean()
        
        # Replace nulls for each label separately
        for label in averages.index:
            csfbiomk1.loc[(csfbiomk1['checkin_bin'] == label) & (csfbiomk1[column].isnull()), column] = averages[label]

print(csfbiomk1.head())

   subject_id  visit  checkin_bin  exposurebin  age_decade  racecat_combined  \
0      1001.0      1            2            1         1.0               5.0   
1      1002.0      1            2            1         1.0               5.0   
2      1003.0      1            2            1         1.0               5.0   
3      1004.0      1            1            1         2.0               5.0   
4      1005.0      1            3            0         2.0               5.0   

   eduyears  totyr_foot  chiiseas_pf  chiiyrs_pf  ...    c_pT181  \
0      16.0         7.0       4335.4      2167.7  ...  24.400000   
1      15.0        14.0      10363.1      5708.1  ...  32.220000   
2      18.0        12.0       6685.4      4863.9  ...  32.220000   
3      16.0        16.0       7701.2      6448.9  ...  39.058000   
4      21.0         NaN          NaN         NaN  ...  38.311628   

   c_tTau_FLAG_below_ref_range  c_FLAG_hemolysis  c_tTau_FLAG_insufficient  \
0                     0.000000  

In [6]:
csfbiomk1.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\csfbiomk1_nullreplaced.csv")
original_data = csfbiomk1.copy()

In [7]:
csfbiomk1 = csfbiomk1.drop(columns=csfbiomk1.loc[:,'subject_id':'chiiyrs_pg'].columns)
csfbiomk1 = csfbiomk1.drop(columns='c_strem2_FLAG_insufficient')
csfbiomk1 = csfbiomk1.drop(columns=['c_PDGFRbeta_FLAG_insufficient',
                              'c_tTau_FLAG_below_ref_range',
                              'c_FLAG_hemolysis', 
                              'c_tTau_FLAG_insufficient',
                              'dxcte_ptau217_csf_flag_dv'])
csfbiomk1.info()
csfbiomk1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   c_strem2     236 non-null    float64
 1   c_PDGFRbeta  236 non-null    float64
 2   c_Ab40       236 non-null    float64
 3   c_Ab42       236 non-null    float64
 4   c_pT181      236 non-null    float64
 5   c_ttau       236 non-null    float64
 6   c_NfL        236 non-null    float64
 7   c_GFAP       236 non-null    float64
 8   c_pT231      236 non-null    float64
 9   c_pT217      236 non-null    float64
dtypes: float64(10)
memory usage: 18.6 KB


,c_strem2,c_PDGFRbeta,c_Ab40,c_Ab42,c_pT181,c_ttau,c_NfL,c_GFAP,c_pT231,c_pT217
0,2035.000000,295.000000,8046.000000,702.000000,24.400000,598.000000,373.960000,8706.690000,487.340000,0.605900
1,2683.750000,344.650000,10146.975000,863.500000,32.220000,401.702703,735.907250,8163.845946,501.851892,1.254549
2,2683.750000,344.650000,10146.975000,863.500000,32.220000,401.702703,735.907250,8163.845946,501.851892,1.254549
3,2397.612245,361.773196,9599.500000,790.090000,39.058000,418.709677,780.569900,9070.057586,614.922874,2.080534
4,2752.976744,410.348837,11663.651163,964.255814,38.311628,393.948718,756.707209,8165.744286,525.304762,1.457107


In [8]:
csfbiomk1.head(10)

,c_strem2,c_PDGFRbeta,c_Ab40,c_Ab42,c_pT181,c_ttau,c_NfL,c_GFAP,c_pT231,c_pT217
0,2035.000000,295.000000,8046.000000,702.000000,24.400000,598.000000,373.960000,8706.690000,487.340000,0.605900
1,2683.750000,344.650000,10146.975000,863.500000,32.220000,401.702703,735.907250,8163.845946,501.851892,1.254549
2,2683.750000,344.650000,10146.975000,863.500000,32.220000,401.702703,735.907250,8163.845946,501.851892,1.254549
3,2397.612245,361.773196,9599.500000,790.090000,39.058000,418.709677,780.569900,9070.057586,614.922874,2.080534
4,2752.976744,410.348837,11663.651163,964.255814,38.311628,393.948718,756.707209,8165.744286,525.304762,1.457107
5,2683.750000,344.650000,10146.975000,863.500000,32.220000,401.702703,735.907250,8163.845946,501.851892,1.254549
6,2397.612245,361.773196,9599.500000,790.090000,39.058000,418.709677,780.569900,9070.057586,614.922874,0.977400
7,4626.000000,391.000000,11797.000000,936.000000,40.000000,310.000000,1075.590000,37506.300000,609.430000,0.481200
8,2397.612245,361.773196,9599.500000,790.090000,39.058000,418.709677,780.569900,9070.057586,614.922874,2.080534
9,2683.750000,344.650000,10146.975000,863.500000,32.220000,401.702703,735.907250,8163.845946,501.851892,1.254549


In [9]:
# Calculate the mean for each column, ignoring NaNs
mean_values = csfbiomk1.mean()

csfbiomk1 = csfbiomk1.fillna(mean_values)

csfbiomk1.head()

,c_strem2,c_PDGFRbeta,c_Ab40,c_Ab42,c_pT181,c_ttau,c_NfL,c_GFAP,c_pT231,c_pT217
0,2035.000000,295.000000,8046.000000,702.000000,24.400000,598.000000,373.960000,8706.690000,487.340000,0.605900
1,2683.750000,344.650000,10146.975000,863.500000,32.220000,401.702703,735.907250,8163.845946,501.851892,1.254549
2,2683.750000,344.650000,10146.975000,863.500000,32.220000,401.702703,735.907250,8163.845946,501.851892,1.254549
3,2397.612245,361.773196,9599.500000,790.090000,39.058000,418.709677,780.569900,9070.057586,614.922874,2.080534
4,2752.976744,410.348837,11663.651163,964.255814,38.311628,393.948718,756.707209,8165.744286,525.304762,1.457107


In [10]:
# calling Neuropsychiatric Measurements

briefa_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\briefa_20231117.csv"
briefa = pd.read_csv(briefa_file_path)
briefa = pd.DataFrame(briefa)


In [11]:
# Check for null values in each column
briefa_nulls = briefa.isnull().sum()

briefa_nulls_df = pd.DataFrame({
    'Column': briefa_nulls.index,
    'Number of Nulls': briefa_nulls.values
})

briefa_nulls_df_transposed = briefa_nulls_df.T

print("Null values in briefa:")
print(briefa_nulls_df)
briefa_nulls_df_transposed.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\briefa_nulls.csv", index=False)


Null values in briefa:
          Column  Number of Nulls
0     subject_id                0
1          visit                0
2    checkin_bin                0
3    exposurebin                0
4     age_decade                0
..           ...              ...
102      workorg                0
103    upseteasy                0
104    impulsive                0
105       pickup                0
106     complete                0

[107 rows x 2 columns]


In [12]:
briefa.head()

,subject_id,visit,checkin_bin,exposurebin,age_decade,racecat_combined,eduyears,totyr_foot,chiiseas_pf,chiiyrs_pf,...,activityorg,getover,onething,moodchange,consequence,workorg,upseteasy,impulsive,pickup,complete
0,1001,1,2,1,1,5,16.0,7.0,4335.4,2167.7,...,2,1,2,1,1,2,2,1,1,2
1,1002,1,2,1,1,5,15.0,14.0,10363.1,5708.1,...,3,3,3,2,3,3,3,3,2,3
2,1003,1,2,1,1,5,18.0,12.0,6685.4,4863.9,...,3,3,3,3,2,3,3,3,1,2
3,1004,1,1,1,2,5,16.0,16.0,7701.2,6448.9,...,1,1,1,1,1,1,2,1,1,1
4,1005,1,3,0,2,5,21.0,NaN,NaN,NaN,...,1,2,1,1,1,1,1,1,1,1


In [13]:
briefa = briefa.drop(columns=briefa.loc[:,'subject_id':'chiiyrs_pg'].columns)
briefa = briefa.drop(columns=briefa.loc[:,'negativ':'complete'].columns)
briefa.head()

,mi,tmi,gec,tgec,bri,tbri,inhibit,shift,emotcont,selfmon,initiate,workmem,planorg,taskmon,orgmat
0,75,64,114,57,39,46,12,9,11,7,14,17,20,13,11
1,101,85,171,85,70,79,17,15,23,15,19,24,27,15,16
2,90,77,158,79,68,77,15,14,27,12,20,22,22,14,12
3,43,40,81,42,38,46,9,8,15,6,9,8,11,7,8
4,43,39,81,41,38,45,9,11,12,6,10,8,10,7,8


In [14]:
briefa.describe()

,mi,tmi,gec,tgec,bri,tbri,inhibit,shift,emotcont,selfmon,initiate,workmem,planorg,taskmon,orgmat
count,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000
mean,63.682203,56.724576,110.169492,56.508475,46.487288,55.203390,12.169492,9.690678,15.758475,8.868644,12.605932,13.758475,15.338983,9.466102,12.512712
std,19.879120,16.771696,33.473344,17.217382,14.657067,16.438698,3.606861,3.378543,5.769042,3.063115,4.101039,4.922909,5.199379,2.946855,4.335387
min,40.000000,36.000000,70.000000,35.000000,30.000000,36.000000,8.000000,6.000000,10.000000,6.000000,8.000000,8.000000,10.000000,6.000000,8.000000
25%,45.000000,41.000000,79.750000,41.000000,34.000000,41.000000,9.000000,6.000000,10.000000,6.000000,9.000000,9.000000,10.000000,6.000000,9.000000
50%,60.000000,53.000000,104.500000,54.000000,43.000000,51.000000,11.000000,9.000000,14.000000,8.000000,12.000000,13.000000,14.000000,9.000000,11.000000
75%,79.250000,68.250000,136.000000,69.000000,57.000000,67.250000,14.000000,12.000000,20.000000,11.000000,16.000000,18.000000,19.250000,12.000000,15.000000
max,116.000000,104.000000,204.000000,108.000000,88.000000,105.000000,23.000000,18.000000,30.000000,18.000000,23.000000,24.000000,29.000000,18.000000,24.000000


In [15]:
csfbiomk1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   c_strem2     236 non-null    float64
 1   c_PDGFRbeta  236 non-null    float64
 2   c_Ab40       236 non-null    float64
 3   c_Ab42       236 non-null    float64
 4   c_pT181      236 non-null    float64
 5   c_ttau       236 non-null    float64
 6   c_NfL        236 non-null    float64
 7   c_GFAP       236 non-null    float64
 8   c_pT231      236 non-null    float64
 9   c_pT217      236 non-null    float64
dtypes: float64(10)
memory usage: 18.6 KB


In [16]:
import os
main_path = r'Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers'
folder_name = 'blood_biomk1_to_briefa'  
save_path = os.path.join(main_path, folder_name)
os.makedirs(save_path, exist_ok=True)


In [17]:
categories_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\categories.csv"
categories_df = pd.read_csv(categories_file_path)
new_column = categories_df['checkin_bin']
briefa['category'] = new_column
print(briefa.head())

csfbiomk1['category'] = new_column
print(csfbiomk1.head())


    mi  tmi  gec  tgec  bri  tbri  inhibit  shift  emotcont  selfmon  \
0   75   64  114    57   39    46       12      9        11        7   
1  101   85  171    85   70    79       17     15        23       15   
2   90   77  158    79   68    77       15     14        27       12   
3   43   40   81    42   38    46        9      8        15        6   
4   43   39   81    41   38    45        9     11        12        6   

   initiate  workmem  planorg  taskmon  orgmat  category  
0        14       17       20       13      11         2  
1        19       24       27       15      16         2  
2        20       22       22       14      12         2  
3         9        8       11        7       8         1  
4        10        8       10        7       8         3  
      c_strem2  c_PDGFRbeta        c_Ab40      c_Ab42    c_pT181      c_ttau  \
0  2035.000000   295.000000   8046.000000  702.000000  24.400000  598.000000   
1  2683.750000   344.650000  10146.975000  863.500000

In [18]:
# csfbiomk1['category'] = briefa['category']

# correlation_matrix = csfbiomk1.corr()

# plt.figure(figsize=(12, 8))
# sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', square=True)
# plt.title('Correlation Heatmap of Features')
# plt.show()

# # Alternatively, scatter plots for each feature vs y (checkin_bin)
# for feature in csfbiomk1.columns[:-1]:  # Exclude the label column
#     plt.figure(figsize=(10, 6))
#     sns.boxplot(x='category', y=feature, data=csfbiomk1)
#     plt.title(f'Boxplot of {feature} by category')
#     plt.xlabel('category')
#     plt.ylabel(feature)
#     plt.show()


In [19]:
csfbiomk1.copy()

,c_strem2,c_PDGFRbeta,c_Ab40,c_Ab42,c_pT181,c_ttau,c_NfL,c_GFAP,c_pT231,c_pT217,category
0,2035.000000,295.000000,8046.000000,702.000000,24.400000,598.000000,373.960000,8706.690000,487.340000,0.605900,2
1,2683.750000,344.650000,10146.975000,863.500000,32.220000,401.702703,735.907250,8163.845946,501.851892,1.254549,2
2,2683.750000,344.650000,10146.975000,863.500000,32.220000,401.702703,735.907250,8163.845946,501.851892,1.254549,2
3,2397.612245,361.773196,9599.500000,790.090000,39.058000,418.709677,780.569900,9070.057586,614.922874,2.080534,1
4,2752.976744,410.348837,11663.651163,964.255814,38.311628,393.948718,756.707209,8165.744286,525.304762,1.457107,3
...,...,...,...,...,...,...,...,...,...,...,...
231,607.000000,279.000000,6679.000000,621.000000,18.000000,401.702703,273.150000,954.370000,343.900000,0.572700,2
232,2231.000000,351.000000,9686.000000,801.000000,26.200000,348.000000,423.170000,3147.720000,340.060000,1.457107,3
233,987.000000,201.000000,5283.000000,536.000000,12.900000,401.702703,220.620000,1691.780000,183.410000,0.111200,2
234,3525.000000,477.000000,9057.000000,810.000000,28.400000,186.000000,954.590000,5463.900000,499.050000,2.080534,1



# 95th percentile for both analysis and plotting

In [20]:
for feature in csfbiomk1.columns[:-1]:  # Assuming 'category' is the last column
    upper_limit = csfbiomk1[feature].quantile(0.95)  # Cap at 95th percentile
    lower_limit = csfbiomk1[feature].quantile(0.05)  # Floor at 5th percentile
    csfbiomk1[feature] = np.where(csfbiomk1[feature] > upper_limit, upper_limit, csfbiomk1[feature])
    csfbiomk1[feature] = np.where(csfbiomk1[feature] < lower_limit, lower_limit, csfbiomk1[feature])

    

# Initial Step for Trainings

In [21]:
X = csfbiomk1.copy()

In [22]:
X

,c_strem2,c_PDGFRbeta,c_Ab40,c_Ab42,c_pT181,c_ttau,c_NfL,c_GFAP,c_pT231,c_pT217,category
0,2035.000000,295.000000,8046.000000,702.000000,24.400000,598.000000,373.960000,8706.690000,487.340000,0.605900,2
1,2683.750000,344.650000,10146.975000,863.500000,32.220000,401.702703,735.907250,8163.845946,501.851892,1.254549,2
2,2683.750000,344.650000,10146.975000,863.500000,32.220000,401.702703,735.907250,8163.845946,501.851892,1.254549,2
3,2397.612245,361.773196,9599.500000,790.090000,39.058000,418.709677,780.569900,9070.057586,614.922874,2.080534,1
4,2752.976744,410.348837,11663.651163,964.255814,38.311628,393.948718,756.707209,8165.744286,525.304762,1.457107,3
...,...,...,...,...,...,...,...,...,...,...,...
231,1109.750000,279.000000,6679.000000,621.000000,19.575000,401.702703,273.150000,1900.257500,343.900000,0.572700,2
232,2231.000000,351.000000,9686.000000,801.000000,26.200000,348.000000,423.170000,3147.720000,340.060000,1.457107,3
233,1109.750000,208.750000,5874.500000,536.000000,19.575000,401.702703,272.692500,1900.257500,278.202500,0.318225,2
234,3525.000000,477.000000,9057.000000,810.000000,28.400000,189.500000,954.590000,5463.900000,499.050000,2.080534,1


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# got error due to non numeric values, so removing them:

# Assume csfbiomk1 is your DataFrame loaded with various types of data
X = csfbiomk1.copy()

# Select only numeric columns from the DataFrame
X_numeric = X.select_dtypes(include=[np.number])
X = X_numeric

# Now X_numeric contains only the numeric columns of the original DataFrame
# Proceed with scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and testing sets
X_train, X_test = train_test_split(X_scaled, test_size=0.3, random_state=42)


models = {}
predictions = {}


X_train, X_test = train_test_split(X_scaled, test_size=0.3, random_state=42)

In [24]:
# Define the columns you want to select for modeling
## columns = ['mi', 'tmi','gec','tgec','bri','tbri','inhibit','shift','emotcont','selfmon','initiate','workmem','planorg','taskmon','orgmat'] 
selected_columns = ['orgmat']  # Replace 'column1' and 'column2' with the actual column names

# selected_columns = ['mi', 'tmi','gec','tgec','bri','tbri','inhibit','shift','emotcont','selfmon','initiate','workmem','planorg','taskmon','orgmat']  # Replace 'column1' and 'column2' with the actual column names


In [25]:
selected_columns

['orgmat']

In [26]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

# Initialize an empty list to store results
results = []
feature_importance_rankings = {}
model_name = "Random Forest"
selected_columns_str = "_".join(selected_columns)

# First, calculate feature importances for each target column and save rankings
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Train full model to get feature importances
    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train)
    
    # Get feature importance and rank features
    feature_importance = pd.Series(model.feature_importances_, index=X_train.columns)
    ranked_features = feature_importance.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Save performance metrics with all features as a baseline
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_csfbiomk1_rank_{selected_columns_str}.xlsx") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")


# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = RandomForestRegressor(random_state=42)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_csfbiomk1_w_top_features_{selected_columns_str}.xlsx") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to feature_importance_rankings.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")



Feature importance rankings saved to feature_importance_rankings.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [27]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Multiple Linear Regression"
selected_columns_str = "_".join(selected_columns)

# Standardize features for linear regression
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# First, calculate feature importances using SelectKBest for each target column and save rankings
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train full model with all features as a baseline
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_csfbiomk1_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = LinearRegression()
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_csfbiomk1_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [28]:
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Gradient Boosting Machines"
selected_columns_str = "_".join(selected_columns)

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# First, calculate feature importances using SelectKBest for each target column and save rankings
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train full model with all features as a baseline
    model = LGBMRegressor(random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_csfbiomk1_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = LGBMRegressor(random_state=42)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_csfbiomk1_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000073 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 407
[LightGBM] [Info] Number of data points in the train set: 165, number of used features: 11
[LightGBM] [Info] Start training from score 12.466667
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000137 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 407
[LightGBM] [Info] Number of data points in the train set: 165, number of used features: 11
[LightGBM] [Info] Start training from score 12.466667
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

In [29]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "XGBoost"
selected_columns_str = "_".join(selected_columns)

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# First, calculate feature importances using SelectKBest for each target column and save rankings
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train full model with all features as a baseline
    model = xgb.XGBRegressor(random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_csfbiomk1_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = xgb.XGBRegressor(random_state=42)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_csfbiomk1_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [30]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Support Vector Regression"
selected_columns_str = "_".join(selected_columns)

# Standardize features for SVR
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Calculate feature importances using SelectKBest for each target column and save rankings
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train SVR model with all features as a baseline
    model = SVR(kernel='rbf', C=0.2, epsilon=0.01)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_csfbiomk1_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = SVR(kernel='rbf', C=0.2, epsilon=0.01)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_csfbiomk1_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [31]:
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "ANN"
selected_columns_str = "_".join(selected_columns)

# Standardize features for ANN
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Feature selection and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train ANN model with all features as a baseline
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    
    model = Sequential([
        Dense(128, activation='relu', input_dim=X_train_selected.shape[1]),
        Dense(64, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train_selected, y_train, epochs=100, batch_size=10, verbose=0)
    
    y_pred = model.predict(X_test_selected).flatten()
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_csfbiomk1_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Scale selected features for ANN
        X_train_selected_scaled = scaler.fit_transform(X_train_selected)
        X_test_selected_scaled = scaler.transform(X_test_selected)
        
        # Train and evaluate ANN model with selected features
        model = Sequential([
            Dense(128, activation='relu', input_dim=X_train_selected_scaled.shape[1]),
            Dense(64, activation='relu'),
            Dense(1)
        ])
        model.compile(optimizer='adam', loss='mse')
        model.fit(X_train_selected_scaled, y_train, epochs=100, batch_size=10, verbose=0)
        
        y_pred = model.predict(X_test_selected_scaled).flatten()
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_csfbiomk1_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


3/3 [==============================] - 0s 997us/step


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


3/3 [==============================] - 0s 998us/step
Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [32]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "KNeighbors Regressor"
selected_columns_str = "_".join(selected_columns)

# Standardize features for KNN
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Feature selection and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train KNN model with all features as a baseline
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    
    model = KNeighborsRegressor(n_neighbors=5)
    model.fit(X_train_selected, y_train)
    
    y_pred = model.predict(X_test_selected)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_csfbiomk1_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate KNN model with selected features
        model = KNeighborsRegressor(n_neighbors=5)
        model.fit(X_train_selected, y_train)
        
        y_pred = model.predict(X_test_selected)
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_csfbiomk1_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [33]:
import pandas as pd
from sklearn.ensemble import StackingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Stacking Regressor"
selected_columns_str = "_".join(selected_columns)

# Define the base models and final estimator for stacking
base_models = [
    ('knn', KNeighborsRegressor(n_neighbors=13, leaf_size=23, p=1, weights='uniform')),
    ('svr', SVR(kernel='rbf', C=0.2, epsilon=0.01))
]
final_estimator = Ridge()
stack_model = StackingRegressor(estimators=base_models, final_estimator=final_estimator, cv=5)

# Feature selection and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train stacking model with all features as a baseline
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    
    stack_model.fit(X_train_selected, y_train)
    y_pred = stack_model.predict(X_test_selected)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_csfbiomk1_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate stacking model with selected features
        stack_model.fit(X_train_selected, y_train)
        y_pred = stack_model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_csfbiomk1_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [34]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Bagging with SVR"
selected_columns_str = "_".join(selected_columns)

# Define the base model for Bagging
base_model = SVR(kernel='rbf', C=0.2, epsilon=0.01)

# Feature selection and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train Bagging model with all features as a baseline
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    
    bagging_model = BaggingRegressor(base_estimator=base_model, n_estimators=10, random_state=42, n_jobs=-1)
    bagging_model.fit(X_train_selected, y_train)
    y_pred = bagging_model.predict(X_test_selected)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_csfbiomk1_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate Bagging model with selected features
        bagging_model = BaggingRegressor(base_estimator=base_model, n_estimators=10, random_state=42, n_jobs=-1)
        bagging_model.fit(X_train_selected, y_train)
        y_pred = bagging_model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_csfbiomk1_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:1

Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [35]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Bagging with SVR"
selected_columns_str = "_".join(selected_columns)

# Define the base model for Bagging
base_model = SVR(kernel='rbf', C=0.2, epsilon=0.01)

# Feature selection and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train Bagging model with all features as a baseline
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    
    bagging_model = BaggingRegressor(base_estimator=base_model, n_estimators=10, random_state=42, n_jobs=-1)
    bagging_model.fit(X_train_selected, y_train)
    y_pred = bagging_model.predict(X_test_selected)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_csfbiomk1_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate Bagging model with selected features
        bagging_model = BaggingRegressor(base_estimator=base_model, n_estimators=10, random_state=42, n_jobs=-1)
        bagging_model.fit(X_train_selected, y_train)
        y_pred = bagging_model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_csfbiomk1_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:1

Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [36]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Lasso Regression"
selected_columns_str = "_".join(selected_columns)

# Standardize features for Lasso regression
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Feature ranking and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Lasso Regression Model with L1 regularization for feature selection
    model = Lasso(alpha=0.1)
    model.fit(X_train, y_train)
    
    # Select features based on non-zero coefficients
    selector = SelectFromModel(model, prefit=True)
    selected_features = X_train.columns[selector.get_support()]
    
    # Feature importance ranking based on absolute Lasso coefficients
    feature_importance = pd.Series(abs(model.coef_), index=X_train.columns)
    ranked_features = feature_importance.sort_values(ascending=False)
    feature_importance_rankings[column] = ranked_features

    # Evaluate baseline performance with all features
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_csfbiomk1_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select top `n_features` based on ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = Lasso(alpha=0.1)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_csfbiomk1_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [37]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel

# Initialize lists and dictionaries to store results and feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Ridge Regression"
selected_columns_str = "_".join(selected_columns)

# Standardize features for Ridge regression
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Feature ranking and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Ridge Regression Model
    model = Ridge(alpha=1.0)
    model.fit(X_train, y_train)
    
    # Use coefficients for feature importance
    feature_importance = pd.Series(abs(model.coef_), index=X_train.columns)
    ranked_features = feature_importance.sort_values(ascending=False)
    feature_importance_rankings[column] = ranked_features
    
    # Evaluate baseline performance with all features
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_csfbiomk1_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select top `n_features` based on ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = Ridge(alpha=1.0)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_csfbiomk1_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [38]:
import pandas as pd
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Initialize lists and dictionaries for storing results and feature rankings
results = []
feature_importance_rankings = {}
model_name = "Bayesian Ridge"
selected_columns_str = "_".join(selected_columns)

# Standardize features for consistency
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Feature ranking and baseline model training
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Train full model to get feature importances
    model = BayesianRidge()
    model.fit(X_train, y_train)
    
    # Use coefficients for feature importance ranking
    feature_importance = pd.Series(abs(model.coef_), index=X_train.columns)
    ranked_features = feature_importance.sort_values(ascending=False)
    feature_importance_rankings[column] = ranked_features
    
    # Baseline performance with all features
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_csfbiomk1_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select top `n_features` based on ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = BayesianRidge()
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_csfbiomk1_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [39]:
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Gaussian Process Regression"
selected_columns_str = "_".join(selected_columns)

# Define the kernel
kernel = RBF(1.0)

# Standardize features for Gaussian Process Regression
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# First, calculate feature importances using SelectKBest for each target column and save rankings
for column in selected_columns:
    y = briefa[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train the full Gaussian Process model as a baseline
    model = GaussianProcessRegressor(kernel=kernel, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluate and store baseline performance with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_csfbiomk1_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = briefa[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = GaussianProcessRegressor(kernel=kernel, random_state=42)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_csfbiomk1_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
C:\Users\fargor\AppData\Lo

Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx
